In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 35.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 58.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 14.4 MB/s eta 0:00:00


In [1]:
from transformers import AutoTokenizer, AutoModelWithLMHead
import torch

tokenizer = AutoTokenizer.from_pretrained("pierreguillou/gpt2-small-portuguese")
model = AutoModelWithLMHead.from_pretrained("pierreguillou/gpt2-small-portuguese")

# define o tamanho máximo da sentença para 1024 caracteres
tokenizer.model_max_length=1024

model.eval()  # desabilita dropout (estamos utilizando apenas como execução do modelo já pré-treinado)


tokenizer_config.json:   0%|          | 0.00/92.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/666 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/850k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/508k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/120 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/modeling_auto.py:1509: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/510M [00:00<?, ?B/s]

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

## Gerando uma palavra

In [2]:
# input sequence
text = "Quem foi Machado de Assis? Machado de Assis foi um"

In [3]:
inputs = tokenizer(text, return_tensors="pt")

# model output
outputs = model(**inputs, labels=inputs["input_ids"])

In [4]:
loss, logits = outputs[:2]
print("loss: ", loss)

predicted_index = torch.argmax(logits[0, -1, :]).item()
predicted_text = tokenizer.decode([predicted_index])

# results
print('input text:', text)
print('predicted text:', predicted_text)

loss:  tensor(3.2219, grad_fn=<NllLossBackward0>)
input text: Quem foi Machado de Assis? Machado de Assis foi um
predicted text:  dos


## Gerando uma sentença completa

### Top-k

In [6]:
inputs = tokenizer(text, return_tensors="pt")

# Saída do modelo utilizando a amostragem Top-k para geração do texto
sample_outputs = model.generate(inputs.input_ids,
                                pad_token_id=50256,
                                do_sample=True,
                                max_length=50,
                                top_k=40,
                                num_return_sequences=1)

# generated sequence
for i, sample_output in enumerate(sample_outputs):
    print(">> Generated text {}\n\n{}".format(i+1, tokenizer.decode(sample_output.tolist())))

>> Generated text 1

Quem foi Machado de Assis? Machado de Assis foi um escritor brasileiro, tradutor, contista e jornalista que escreveu vários romances nos campos das artes, que ele mesmo chamou de "histórias de Machado de Assis". Para além de seus livros que se tornaram


### Top-p

In [9]:
inputs = tokenizer(text, return_tensors="pt")

# Saída do modelo utilizando a amostragem Top-k para geração do texto
sample_outputs = model.generate(inputs.input_ids,
                                pad_token_id=50256,
                                do_sample=True,
                                max_length=50,
                                top_p=0.6,
                                num_return_sequences=1)

# generated sequence
for i, sample_output in enumerate(sample_outputs):
    print(">> Generated text {}\n\n{}".format(i+1, tokenizer.decode(sample_output.tolist())))

>> Generated text 1

Quem foi Machado de Assis? Machado de Assis foi um dos fundadores do Colégio dos Jesuítas, no Rio de Janeiro, em 1647.

A sua obra foi publicada no ano de 1653, no "Boletim do Colégio dos Jesuítas" e


### Temperatura

In [15]:
inputs = tokenizer(text, return_tensors="pt")

# Saída do modelo utilizando a amostragem Top-k para geração do texto
sample_outputs = model.generate(inputs.input_ids,
                                pad_token_id=50256,
                                do_sample=True,
                                max_length=50,
                                temperature=0.00001,
                                num_return_sequences=1)

# generated sequence
for i, sample_output in enumerate(sample_outputs):
    print(">> Generated text {}\n\n{}".format(i+1, tokenizer.decode(sample_output.tolist())))

>> Generated text 1

Quem foi Machado de Assis? Machado de Assis foi um dos mais importantes escritores da história brasileira.

A obra de Machado de Assis é uma das mais importantes obras de ficção brasileira. A obra de Machado de Assis é uma das mais importantes obras de


### Todos os métodos de amostragem

In [17]:
inputs = tokenizer(text, return_tensors="pt")

# Saída do modelo utilizando a amostragem Top-k para geração do texto
sample_outputs = model.generate(inputs.input_ids,
                                pad_token_id=50256,
                                do_sample=True,
                                max_length=50,
                                top_p=0.6,
                                temperature=0.00001,
                                top_k=50,
                                num_return_sequences=1)

# generated sequence
for i, sample_output in enumerate(sample_outputs):
    print(">> Generated text {}\n\n{}".format(i+1, tokenizer.decode(sample_output.tolist())))

>> Generated text 1

Quem foi Machado de Assis? Machado de Assis foi um dos mais importantes escritores da história brasileira.

A obra de Machado de Assis é uma das mais importantes obras de ficção brasileira. A obra de Machado de Assis é uma das mais importantes obras de


# Utilizando a API da OpenAI para o GPT-3

In [ ]:
!pip install openai

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 KB 2.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for openai: filename=openai-0.26.5-py3-none-any.whl size=67620 sha256=8a4667febc20a7da31a98efb440470f806729cfdbd6e173b0e18d0baef8a2724
  Stored in directory: /root/.cache/pip/wheels/a7/47/99/8273a59fbd59c303e8ff175416d5c1c9c03a2e83ebf7525a99
Successfully built openai


In [ ]:
import openai

# Set up the OpenAI API client
openai.api_key = "YOUR_API_KEY"

# Set up the model and prompt
model_engine = "text-davinci-003"
prompt = "Hello, how are you today?"

# Generate a response
completion = openai.Completion.create(
    engine=model_engine,
    prompt=prompt,
    max_tokens=1024,
    n=1,
    stop=None,
    temperature=0.5,
)

response = completion.choices[0].text
print(response)